In [ ]:

import wandb
from wandb.keras import WandbCallback

# Essentials
import numpy as np
from matplotlib import pyplot
import tensorflow
from tensorflow import keras
from keras import regularizers
from keras.models import Sequential
from keras.utils import np_utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3, ResNet50, InceptionResNetV2, Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation
from keras.callbacks import  Callback, EarlyStopping

import random
import imageio
import os
import cv2
import glob
random.seed(42)



In [ ]:
# !wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
# !unzip -q '/content/nature_12K.zip'

# Define the labels for the Simpsons characters we're detecting
class_names = {0:'Amphibia', 1:'Animalia', 2:'Arachnida',3: 'Aves',4: 'Fungi',
              5: 'Insecta', 6:'Mammalia', 7:'Mollusca', 8:'Plantae',9: 'Reptilia'}

num_classes = 10
img_size = 128
dir = '/content/inaturalist_12K/train'

datagen = ImageDataGenerator(
    rotation_range=45,  
    width_shift_range=0.2, 
    height_shift_range=0.3, 
    horizontal_flip=True, 
    vertical_flip=True,  
    validation_split=0.2 
)


input_shape = (224,224,3)


In [ ]:
sweep_config = {
  "metric": {
      "name":"val_accuracy",
      "goal": "maximize"
  },
  "method": "random",
    
  "parameters": {
          
        "nn" :{"values": ['Inception3','InceptionResNetV2', 'ResNet50', 'xception']},


        "data_aug": {
            "values": [True, False]
        },
        "train_batch_size": {
            "values": [32]
        },
        "dropout": {
            "values": [0, 0.2, 0.1]
        },
        "fc_layer": {
            "values": [128, 256, 512]
        },
        "pre_layer_train": {
            "values": [30]
        },
        "epochs":{
            "values": [5,10]
        }
    }
}

In [ ]:
def define_model(pretrained_model_name, activation_function_dense, fc_layer, dropout, pre_layer_train=None):
    input_image_shape = (img_size, img_size, 3)

    input_ = K.Input(shape = input_image_shape)

    # add a pretrained model without the top dense layer
    if pretrained_model_name == 'ResNet50':
      pretrained_model = K.applications.ResNet50(include_top = False, weights='imagenet',input_tensor = input_)
    elif pretrained_model_name == 'InceptionV3':
      pretrained_model = K.applications.InceptionV3(include_top = False, weights='imagenet',input_tensor = input_)
    elif pretrained_model_name == 'InceptionResNetV2':
      pretrained_model = K.applications.InceptionResNetV2(include_top = False, weights='imagenet',input_tensor = input_)
    else:
      pretrained_model = K.applications.Xception(include_top = False, weights='imagenet',input_tensor = input_)
    
    #freeze all layers
    for layer in pretrained_model.layers:
        layer.trainable=False 
    
    #set some of the top layers as trainable
    if pre_layer_train:
      for layer in pretrained_model.layers[-pre_layer_train:]:
        layer.trainable=True

    model = K.models.Sequential()
    model.add(pretrained_model)#add pretrained model
    model.add(Flatten()) # The flatten layer is essential to convert the feature map into a column vector
    model.add(Dense(fc_layer, activation=activation_function_dense))#add a dense layer
    model.add(Dropout(dropout)) # For dropout
    model.add(Dense(10, activation="softmax"))#softmax layer

    return model

In [ ]:


def train(config = sweep_config):

    with wandb.init(project="Assignment2", entity="cs6910jananiteam", config=config):
      config = wandb.init().config
    
      data_aug = config.data_aug
      train_batch_size = config.train_batch_size
      dropout = config.dropout
      fc_layer = config.fc_layer
      pre_layer_train = config.pre_layer_train
      epochs = config.epochs
      pre_train_model = config.nn
      activation_function_dense = config.dense
      
      # Display the hyperparameters
      # Display the hyperparameters
      run_name = "model_{}_aug_{}_bs_{}_drop_{}_fc_{}_fre_{}_epoc_{}".format(pre_train_model, data_aug, train_batch_size, dropout, fc_layer, pre_layer_train, epochs )

      base_model = define_model(pretrained_model_name=pre_train_model, activation_function_dense=activation_function_dense, fc_layer=fc_layer, dropout=dropout, pre_layer_train=pre_layer_train)


      if config.pre_train_layer == 0:
        base_model.trainable = False

      stack_model = Sequential()
      stack_model.add(base_model)
      stack_model.add(Flatten())
      stack_model.add(Dense(config.num_dense, activation='relu'))
      stack_model.add(Dense(10, activation='softmax'))

      
      
      final_model = stack_model
      final_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])


      TrainData = datagen.flow_from_directory(
        directory='/content/inaturalist_12K/train',
        target_size = (224, 224),
        batch_size=config.batch_size,
        class_mode="categorical",
        shuffle=True,
        subset='training',
        seed=123)

      ValidationData = datagen.flow_from_directory(
              directory='/content/inaturalist_12K/train',
              target_size = (224, 224),
              batch_size=config.batch_size,
              class_mode="categorical",
              shuffle=True,
              subset='validation',
              seed=123)
      


      if config.pre_train_layer == 0:

        final_model.fit_generator(
            TrainData,
            steps_per_epoch = TrainData.samples // config.batch_size,
            epochs = config.epochs,
            verbose = 1,
            validation_data = ValidationData,
            validation_steps = ValidationData.samples // config.batch_size,
            callbacks = [WandbCallback(training_data = TrainData,log_weights=(True), log_gradients=(True))]
        )      

      if config.pre_train_layer > 0:
        base_model.trainable = True

        # Freeze layers
        freeze_point = len(base_model.layers) - config.freeze_before
        for layer in base_model.layers[:freeze_point]:
            layer.trainable = False
        
        final_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

        final_model.fit_generator(
            TrainData,
            steps_per_epoch = TrainData.samples // config.batch_size,
            epochs = config.epochs,
            verbose = 1,
            validation_data = ValidationData,
            validation_steps = ValidationData.samples // config.batch_size,
            callbacks = [WandbCallback(training_data = TrainData,log_weights=(True), log_gradients=(True))]
        )      



      



In [ ]:
SweepId = wandb.sweep(sweep_config,project="Assignment2PartB", entity="cs6910jananiteam")
wandb.agent(SweepId, function = train, count = 10)